In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Sentiment Analysis/emoevent_es.csv'
df = pd.read_csv(file_path, delimiter='\t')
label_encoder = LabelEncoder()
df['emotion'] = label_encoder.fit_transform(df['emotion'])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized = tokenizer(df['tweet'].tolist(), max_length=128, padding='max_length', truncation=True, return_tensors='tf')

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X = tokenized['input_ids'].numpy()
y = df['emotion'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=32)

Epoch 1/3
211/211 [==============================] - 218s 798ms/step - loss: 1.3737 - accuracy: 0.5112
Epoch 2/3
211/211 [==============================] - 169s 803ms/step - loss: 1.2454 - accuracy: 0.5695
Epoch 3/3
211/211 [==============================] - 170s 804ms/step - loss: 1.1204 - accuracy: 0.6105


In [ ]:
import numpy as np

In [ ]:
predictions = model.predict(X_test).logits
predicted_labels = [np.argmax(p) for p in predictions]
print(classification_report(y_test, predicted_labels))

53/53 [==============================] - 18s 267ms/step
              precision    recall  f1-score   support

           0       0.24      0.71      0.36       180
           1       0.00      0.00      0.00        30
           2       0.00      0.00      0.00        13
           3       0.61      0.42      0.50       332
           4       0.75      0.68      0.71       835
           5       0.79      0.59      0.68       229
           6       0.00      0.00      0.00        63

    accuracy                           0.58      1682
   macro avg       0.34      0.34      0.32      1682
weighted avg       0.62      0.58      0.58      1682



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
